In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import sys
from pathlib import Path

sys.path.append("../")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
from choice_learn.datasets.examples import load_tafeng

In [ ]:
df = load_tafeng(as_frame=True)
df.head()

In [ ]:
dataset = load_tafeng(as_frame=False, preprocessing="assort_example")

In [ ]:
dataset.contexts_items_features[0].shape

In [ ]:
dataset.summary()

In [ ]:
import tensorflow as tf
price_elasticities = tf.Variable(
                            tf.random_normal_initializer(0.0, 0.02, seed=42)(shape=(1, 3)) # 3 age categories
                        )

In [ ]:
tf.tensordot(dataset.contexts_features[0][:16].astype("float32"), tf.transpose(price_elasticities),  axes=1)

In [ ]:
dataset.contexts_items_features[0].shape, dataset.contexts_features[0].shape

In [ ]:
from choice_learn.models.base_model import ChoiceModel

In [ ]:
import tensorflow as tf


class TaFengMNL(ChoiceModel):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        # base utilties
        self.base_utilities = tf.Variable(
                            tf.random_normal_initializer(0.0, 0.02, seed=42)(shape=(1, 25)) # 25 items
                        )
        # price elasticities
        self.price_elasticities = tf.Variable(
                            tf.random_normal_initializer(0.0, 0.02, seed=42)(shape=(1, 3)) # 3 age categories
                        )
                        
        self.weights = [self.base_utilities, self.price_elasticities]

    def compute_batch_utility(self,
                        fixed_items_features,
                        contexts_features,
                        contexts_items_features,
                        contexts_items_availabilities,
                        choices):

        # base utilities
        # return tf.multiply(contexts_items_features[:, :, 0], tf.gather(self.price_elasticities, contexts_features[1], axis=1)[0]) + self.base_utilities
        
        price_coeffs = tf.tensordot(contexts_features, tf.transpose(self.price_elasticities),  axes=1)
        return tf.multiply(contexts_items_features[:, :, 0], price_coeffs) + self.base_utilities


In [ ]:
model = TaFengMNL(optimizer="lbfgs", epochs=1000)
history = model.fit(dataset)

In [ ]:
plt.plot(history)

In [ ]:
print(model.weights)